In [90]:
import mysql.connector
import pandas as pd
import numpy as np
import random

In [91]:
def getConnectionFromServer():
    return mysql.connector.connect(
        host='ds50-mysql-do-user-9644544-0.b.db.ondigitalocean.com',
        database='ds50',
        user='ds50',
        password='AVNS_4ybSd0CoPKnCL5F',
        port = '25060')

In [106]:
#Fonction qui renvoie une liste de tag(leur id) unique choisi aléatoirement dans la table TAG.
#Elle sera remplacé par les 3 tags selectionnés par l'utilisateur (information renvoyé par le front de Zackaria)
#Le premier paramètre nombreDeTagChoisiParUtilisateur precise le nombre de tag renvoyé.
#Le deuxième paramètre TableTag correspond à la dataframe de la table TAG extraite de notre base de données MySQL.
def creationListeTag(nombreDeTagChoisiParUtilisateur : int, TableTag : pd.DataFrame) -> np.array:
    tag = TableTag['tag_id'].unique()
    listeTag = np.array([])
    #Potentiellment, les nombres aléatoires générés peuvent être les mêmes.
    #On itère donc le processus jusq'à ce que tous les nombres générés soit différent
    while np.size(listeTag) < nombreDeTagChoisiParUtilisateur: 
        for i in range(nombreDeTagChoisiParUtilisateur):
            listeTag = np.append(listeTag,int(tag[random.randint(0,len(tag)-1)]))
        listeTag = np.unique(listeTag)
    return listeTag

In [100]:
#Fonction qui renvoie un dictionnaire (clef : livre, valeur : tag) des livre unique choisi, dans la table TAGGED, parmis les plus taggé pour les tags selectionnée par la fonction creationListeTag.
#Le premier paramètre, listeTag, contient la liste des tags choisit (générament extrait de la fonction creationListeTag).
#Le deuxième paramètres, nombreDeLivreARetourner, précise le nombre de livre a renvoyé, on en renvoie 3 fois plus pour choisir parmis cela les mieux noté.
#Le troisième paramètre, tableTagged, correspond à la dataframe de la table TAGGED extraite de notre base de données MySQL.
def creationListeLivrePlusTagged(listeTag : np.array, nombreDeLivreARetourner : int, tableTagged : pd.DataFrame) -> dict:
    n = (nombreDeLivreARetourner//len(listeTag))*3
    counter = 0 
    listeIdLivre = {}
    #Potentiellment, le même livre peut faire partie du top 30 le plus tagger dans deux tag différent.
    #On a donc créer la variable counter pour augmenter le nombre de livre selectionner dand chaque tag différent dans le cas où ça arriverai.
    #On procède ensuite à un mélange aléatoire et à un slice pour s'assurer un équiprobabilité de distibution des livres renvoyés dans les différentes catégories.
    while len(listeIdLivre) < (nombreDeLivreARetourner*3):
        for tag in listeTag:
            books = np.array(tableTagged[tableTagged['tag_id'] == tag].nlargest(n+counter, 'count')[['book_id', 'tag_id']])
            for book, tag in books:
                listeIdLivre[str(book)] = tag
        counter += 1
    listeIdLivre = dict(list(listeIdLivre.items())[:nombreDeLivreARetourner*3])  # On slice
    return listeIdLivre


In [94]:
def creationListeLivrePourChaqueTag(idTag : int, dictionnaireLivreTag : dict) -> np.array:
    listeLivrePourUnTag = np.array([])
    for k, val in dictionnaireLivreTag.items(): 
        if idTag == val:
            listeLivrePourUnTag = np.append(listeLivrePourUnTag, int(k))
    return listeLivrePourUnTag

In [109]:
#Fonction qui permet d'établir les 33% des livres les mieux noté parmis la liste passé en argument
def creationListeLivreMieuxNote(listeLivre : np.array, tableInteraction : pd.DataFrame, listTag : np.array, completude : bool) -> np.array:
    if len(listeLivre) % len(listTag) == 0:
        tableInteractionByBook = tableInteraction[tableInteraction["book_id"].isin(list(listeLivre))]
        tableInteractionByBook = tableInteractionByBook.groupby(['book_id']).mean()
        return np.array(tableInteractionByBook.nlargest(len(listeLivre)//3, 'rating').index.values)
    elif completude == True:
        tableInteractionByBook = tableInteraction[tableInteraction["book_id"].isin(list(listeLivre))]
        tableInteractionByBook = tableInteractionByBook.groupby(['book_id']).mean()
        return np.array(tableInteractionByBook.nlargest((len(listeLivre)//3)+(len(listeLivre) % len(listTag)), 'rating').index.values)
    tableInteractionByBook = tableInteraction[tableInteraction["book_id"].isin(list(listeLivre))]
    tableInteractionByBook = tableInteractionByBook.groupby(['book_id']).mean()
    return np.array(tableInteractionByBook.nlargest((len(listeLivre)//3), 'rating').index.values)
        

In [112]:
#Fonction qui permet d'avoir le dictionnaire des livres à renvoyer à zackaria.
#Le premier argument permet d'indiquer le nombre de livre voulu.
#Le second argument permet d'indiquer le nombre de tag choisi par l'utilisateur.
#Les tags sont determiné aléatoirement par la fonction creationListeTag, elle sera changé avec les informations du front.
def getBookByTagSelected(nombreDeLivreVoulu : int, nombreDeTagSelectionnéParUtilisateur : int) -> dict:
        connection = getConnectionFromServer()
        listeLivrePourChaqueTag = []
        listeLivreFinal = []
        tableTag = pd.read_sql( 'SELECT * FROM TAG', connection)
        tableTagged = pd.read_sql( 'SELECT * FROM TAGGED', connection)
        tableBook = pd.read_sql( 'SELECT * FROM BOOK', connection)
        tableInteraction = pd.read_sql( 'SELECT * FROM INTERACTION', connection)
        tags = creationListeTag(nombreDeTagSelectionnéParUtilisateur, tableTag)
        books = creationListeLivrePlusTagged(tags, nombreDeLivreVoulu, tableTagged)
        completude = True
        for tag in tags:
                listeLivrePourChaqueTag = list(creationListeLivrePourChaqueTag(tag, books))
                listeLivreFinal += list(creationListeLivreMieuxNote(listeLivrePourChaqueTag, tableInteraction, tags, completude))
                listeLivrePourChaqueTag = []
                completude = False
        tableBookSelected = tableBook[tableBook['book_id'].isin(listeLivreFinal)]
        connection.close()
        return tableBook.to_dict('records')

In [113]:
result = getBookByTagSelected(50, 3)

c:\Users\antoi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\antoi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\antoi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\antoi\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas